##### Парсинг

In [3]:
import pandas as pd
task = pd.read_excel('task.xlsx', 2)

ids = (task.loc[:,'674850-6']).to_list()
len(ids) # количество документов, которые нужно скачать

718

In [5]:
def ids_to_link(your_id):
    '''
    Функция берет на вход номер документа и выдает склеенную с общей ссылкой полную ссылку на страничку с документом.
    '''
    your_id = str(your_id)
    link = 'https://sozd.duma.gov.ru'
    return link + '/bill/' + your_id

ids[280] = '4689-7' # "испорченный" номер, который распознался как дата => насильно переделываем в стринг

links = list(map(ids_to_link, ids)) # применяем ф-цию ко всему списку номеров документов 
links[60]

'https://sozd.duma.gov.ru/bill/1094316-6'

In [3]:
# Запускаем драйвер
import time
from selenium import webdriver as wb

br = wb.Chrome("/Users/Alina/Desktop/Python/chromedriver") 

In [8]:
def get_docs(link):
    '''
    Функция берет на вход ссылку на страницу с документом, находит гиперссылку для скачивания по тексту гиперссылки и 
    нажимает, т.о. скачивая документ.
    '''
    br.get(link)
    try:
        br.find_element_by_partial_link_text("Текст внесенного законопроекта").click()
    except Exception:  
        br.find_element_by_partial_link_text("Текст законопроекта").click()
    br.implicitly_wait(2)

[get_docs(i) for i in links]

##### Чтение документов

In [16]:
# Сортируем документы по дате скачивания и получаем список названий
import os
search_dir = "C://Users/Alina/Desktop/Python/duma"
os.chdir(search_dir)
files = filter(os.path.isfile, os.listdir(search_dir))
files = [os.path.join(search_dir, f) for f in files] 
files.sort(key=lambda x: os.path.getmtime(x))

for i in range(0, len(files)):
    old = files[i].split('\\')
    new = ids[464:][i] + '.' + old[-1].split('.')[-1]
    os.rename(old[-1], new)
    print(new)

602468-6.doc
777820-7.doc
1058353-7.doc
542772-7.doc
842639-7.doc
804246-6.rtf
914228-6.doc
527167-6.doc
101253-7.doc
574370-6.doc
669214-6.rtf
769142-7.rtf
1007407-6.doc
773716-6.doc
599736-7.doc
691659-7.PDF
658641-7.doc
1093856-6.doc
195436-7.doc
1107451-6.pdf
1061128-7.doc
126922-7.doc
794920-6.doc
386109-7.doc
195446-7.doc
107145-7.doc
223849-7.doc
990129-7.docx
962487-6.rtf
620109-7.doc
862141-6.rtf
801288-6.doc
325651-7.doc
249505-7.rtf
720839-7.doc
689130-7.doc
29838-7.rtf
987567-6.doc
343329-7.rtf
703636-6.rtf
616883-6.doc
492034-6.doc
419163-7.doc
406404-7.docx
336427-7.doc
137528-7.rtf
268821-7.docx
1039776-6.doc
1064636-6.doc
797659-7.rtf
641225-7.rtf
1106041-6.rtf
1130833-6.doc
162100-7.doc
871036-6.rtf
962462-7.docx
682010-7.doc
90277-7.doc
658681-7.doc
225095-7.rtf
1112830-6.rtf
1098691-6.pdf
1048015-6.doc
1035852-6.doc
809044-7.doc
32913-7.doc
883444-7.docx
602614-7.doc
58706-7.doc
337662-7.doc
114033-7.doc
197556-7.doc
546226-7.doc
826412-6.doc
974393-7.doc
1003767-6.d

In [ ]:
# То же самое для другого списка документов
search_dir = "C://Users/Alina/Desktop/Python/resolutions"
os.chdir(search_dir)
files300 = filter(os.path.isfile, os.listdir(search_dir))
files300 = [os.path.join(search_dir, f) for f in files300] 
files300.sort(key=lambda x: os.path.getmtime(x))

for i in range(0, len(files300)):
    old = files300[i].split('\\')
    new = ids[464:][i] + '.' + old[-1].split('.')[-1]
    os.rename(old[-1], new)
    print(new)

In [ ]:
# Открываем word- и rtf-файлы из папки
# pdf не прочитает
import win32com.client
import numpy as np
word = win32com.client.Dispatch("Word.Application")
word.visible = False

def get_full_doc(doc:str, path: str):
    doc_path = path + doc
    wb = word.Documents.Open(doc_path)
    doc_active = word.ActiveDocument
    return np.array([doc_active.Range().Text, doc])

full = [get_full_doc(i, 'C:\\Users\\Alina\\Desktop\\Python\\duma\\') for i in files]
full = np.reshape(full, (714, 2)) # каждый элемент - текст и название документа 
full.shape # (714, 2)
#np.save('full.npy', full) можно сохранить, чтобы потом каждый раз не загружать документы, т.к. это долго


full300 = [get_full_doc(i, 'C:\\Users\\Alina\\Desktop\\Python\\resolutions\\') for i in files300]
full300 = np.reshape(full300, (278, 2)) 
full300.shape # (278, 2)
#np.save('full300.npy', full300) 